# Mistral AI SDK - Learning Notebook

This tutorial introduces you to working with the Mistral AI API using the Mistral SDK, progressing from simple API calls to building a CV scoring and retrieval system.

## Learning Objectives

By the end of this tutorial, you will be able to:
- Use the Mistral AI API to generate content
- Build a CV scoring system that evaluates resumes against job descriptions
- Implement RAG-based CV retrieval using vector similarity search
- Combine fast retrieval (FAISS) with intelligent re-ranking (Mistral)

## Prerequisites

Before starting, make sure you have:
- Obtained a Mistral AI API key from [Mistral AI Console](https://console.mistral.ai/)
- Installed the required dependencies listed in `pyproject.toml` via `uv sync`
- Set up your `.env` file with `MISTRAL_API_KEY=your_key_here`

## Part 1: Getting Started with Mistral AI SDK

### Import the SDK and Setup

In [1]:
from mistralai import Mistral
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables from .env file
load_dotenv()

True

### Initialize the Mistral Client

The Mistral SDK uses a `Mistral` client object to make requests. The client handles authentication using your API key.

In [2]:
# Initialize the Mistral client
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("MISTRAL_API_KEY not found in environment. Please set it in .env file.")

client = Mistral(api_key=api_key)
print("Mistral client initialized successfully!")

Mistral client initialized successfully!


### Run Your First Prompt

Let's start with a simple text generation request. The `mistral-large-latest` model is Mistral's most capable model.

In [3]:
# Simple text generation
response = client.chat.complete(
    model="mistral-large-latest",
    messages=[
        {"role": "user", "content": "Explain AI to me like I'm a kid."}
    ]
)

print(response.choices[0].message.content)

Sure! Imagine you have a super-smart robot friend who *loves* to learn—just like how you learn new things in school or from your parents.

### Here’s how AI works in simple steps:
1. **Learning Like a Sponge** 🧽
   - AI starts by watching or reading *tons* of examples (like pictures, words, or games).
   - Example: If you show it 1,000 pictures of cats and dogs, it learns to tell them apart—just like you do!

2. **Practice Makes Perfect** 🎮
   - The AI plays games or solves puzzles *over and over* to get better (like how you practice soccer or math).
   - If it makes a mistake, it says, *"Oops! Let me try again!"*

3. **Helping You!** 🤖
   - Once it learns, it can:
     - **Answer questions** (like a super-smart encyclopedia).
     - **Draw pictures** (like a robot artist).
     - **Play games** (and sometimes beat you at them!).
     - **Talk to you** (like a chat buddy).

### Cool Examples:
- **Voice Assistants**: Like Siri or Alexa—they listen to your voice and do what you ask (e.g.

The response can be rendered directly as markdown in notebooks:

In [4]:
# Display as markdown
display(Markdown(response.choices[0].message.content))

Sure! Imagine you have a super-smart robot friend who *loves* to learn—just like how you learn new things in school or from your parents.

### Here’s how AI works in simple steps:
1. **Learning Like a Sponge** 🧽
   - AI starts by watching or reading *tons* of examples (like pictures, words, or games).
   - Example: If you show it 1,000 pictures of cats and dogs, it learns to tell them apart—just like you do!

2. **Practice Makes Perfect** 🎮
   - The AI plays games or solves puzzles *over and over* to get better (like how you practice soccer or math).
   - If it makes a mistake, it says, *"Oops! Let me try again!"*

3. **Helping You!** 🤖
   - Once it learns, it can:
     - **Answer questions** (like a super-smart encyclopedia).
     - **Draw pictures** (like a robot artist).
     - **Play games** (and sometimes beat you at them!).
     - **Talk to you** (like a chat buddy).

### Cool Examples:
- **Voice Assistants**: Like Siri or Alexa—they listen to your voice and do what you ask (e.g., "Play my favorite song!").
- **Self-Driving Cars**: AI "sees" the road (like you do) and drives safely.
- **Video Games**: Enemies in games sometimes use AI to act smart (like hiding or chasing you).

### But Remember:
- AI isn’t *alive*—it’s just really good at following rules and learning from examples.
- It doesn’t have feelings or dreams (yet!), but it can *pretend* to be friendly.

**Fun Fact**: AI is like a parrot 🦜—it repeats what it’s taught, but it doesn’t *understand* things the way you do!

Want to try teaching an AI? You could show it drawings and say, "This is a dinosaur!" and it might learn to draw one too! 🦖✏️

Does that make sense? 😊

### Start a Chat Conversation

Mistral supports multi-turn conversations where the conversation history is maintained. Each message includes the full conversation context.

In [5]:
# Start a conversation
messages = [
    {"role": "user", "content": "Hello! My name is Alex."}
]

response = client.chat.complete(
    model="mistral-large-latest",
    messages=messages
)

print(response.choices[0].message.content)

Hello, Alex! 😊 It's nice to meet you! How can I assist you today? Whether you have questions, need help with something, or just want to chat, I'm here for it. 🚀


In [6]:
# Continue the conversation - add previous messages to maintain context
messages.append({"role": "assistant", "content": response.choices[0].message.content})
messages.append({"role": "user", "content": "What is my name?"})

response = client.chat.complete(
    model="mistral-large-latest",
    messages=messages
)

print(response.choices[0].message.content)

Your name is **Alex**—nice to officially meet you again! 😄 Let me know if you'd like me to remember anything else about you.


### Control Generation Parameters

You can control the model's behavior using parameters like `temperature` and `max_tokens`.

In [7]:
# Example with different parameters
response = client.chat.complete(
    model="mistral-large-latest",
    temperature=0.7,  # Control randomness (0.0 = deterministic, 1.0 = creative)
    max_tokens=500,  # Limit response length
    messages=[
        {"role": "user", "content": "Write a short haiku about programming."}
    ]
)

print(response.choices[0].message.content)

Debugging at dawn—
a missing semicolon
breaks the whole world.


## Part 2: CV Scoring Project

In this section, we'll build a system that scores a CV against a job description using Mistral's reasoning capabilities.

### Load a Sample CV

Let's load a CV from the CVs folder to use for scoring.

In [8]:
# Load a sample CV
cv_path = "../CVs/Topic_1/01_en.md"

with open(cv_path, 'r', encoding='utf-8') as f:
    cv_content = f.read()

print(f"CV loaded: {len(cv_content)} characters")
print("\nFirst 500 characters:")
print(cv_content[:500])

CV loaded: 2007 characters

First 500 characters:
# Ahmed Al-Otaibi
**AI Engineer**

Riyadh, Saudi Arabia | +966500000000 | ahmed.otaibi@email.com
[linkedin.com/in/ahmed-ai] | [github.com/ahmed-code]

---

## Skills

* **Programming Languages:** Python, C++, SQL.
* **Machine & Deep Learning:** PyTorch, TensorFlow, Scikit-learn, Keras, Hugging Face (Transformers).
* **Deployment & Cloud Computing:** Docker, Kubernetes, AWS (SageMaker), FastAPI, CI/CD Pipelines.
* **Specializations:** NLP (Natural Language Processing), Computer Vision, Generative


### Define a Job Description

Create a sample job description to score the CV against.

In [9]:
job_description = """
AI Engineer Position

We are looking for an experienced AI Engineer to join our team.

Requirements:
- Strong experience with Python and machine learning frameworks (PyTorch, TensorFlow)
- Experience deploying ML models to production (Docker, FastAPI, AWS)
- Background in NLP or Computer Vision
- Experience with Hugging Face transformers
- Strong problem-solving skills and ability to work in a team

Nice to have:
- Experience with Kubernetes
- Knowledge of model optimization techniques (quantization, pruning)
- Experience with recommendation systems
"""

print(job_description)


AI Engineer Position

We are looking for an experienced AI Engineer to join our team.

Requirements:
- Strong experience with Python and machine learning frameworks (PyTorch, TensorFlow)
- Experience deploying ML models to production (Docker, FastAPI, AWS)
- Background in NLP or Computer Vision
- Experience with Hugging Face transformers
- Strong problem-solving skills and ability to work in a team

Nice to have:
- Experience with Kubernetes
- Knowledge of model optimization techniques (quantization, pruning)
- Experience with recommendation systems



### Create Scoring Prompt

Build a prompt that asks Mistral to score the CV against the job description and provide detailed reasoning.

In [10]:
scoring_prompt = f"""You are an expert recruiter evaluating a CV against a job description.

Job Description:
{job_description}

CV:
{cv_content}

Please evaluate this CV and provide:
1. An overall match score from 0-100
2. Detailed reasoning for the score
3. Key strengths that match the job requirements
4. Areas where the candidate falls short
5. Specific examples from the CV that support your evaluation

Format your response as a structured evaluation."""

### Call Mistral API to Score the CV

Now let's send the prompt to Mistral and get the scoring results.

In [11]:
# Get scoring from Mistral
response = client.chat.complete(
    model="mistral-large-latest",
    max_tokens=2048,
    messages=[
        {"role": "user", "content": scoring_prompt}
    ]
)

scoring_result = response.choices[0].message.content
print(scoring_result)

### **Structured Evaluation of Ahmed Al-Otaibi’s CV Against the AI Engineer Job Description**

---

### **1. Overall Match Score: 92/100**
**Rationale:** Ahmed’s CV demonstrates a **strong alignment** with the job requirements, covering nearly all mandatory and several "nice-to-have" skills. His experience in **NLP, Computer Vision, model deployment, and optimization** directly addresses the core needs of the role. Minor gaps (e.g., depth in recommendation systems) prevent a perfect score.

---

### **2. Detailed Reasoning for the Score**

#### **A. Mandatory Requirements (Weight: 70%)**
| **Requirement**               | **Match (0-10)** | **Supporting Evidence**                                                                 | **Score** |
|-------------------------------|------------------|----------------------------------------------------------------------------------------|-----------|
| **Python + ML Frameworks**    | 10/10            | Lists PyTorch, TensorFlow, Keras, Scikit-le

### Display Results as Markdown

Let's render the scoring results nicely:

In [12]:
# Display formatted results
display(Markdown(scoring_result))

### **Structured Evaluation of Ahmed Al-Otaibi’s CV Against the AI Engineer Job Description**

---

### **1. Overall Match Score: 92/100**
**Rationale:** Ahmed’s CV demonstrates a **strong alignment** with the job requirements, covering nearly all mandatory and several "nice-to-have" skills. His experience in **NLP, Computer Vision, model deployment, and optimization** directly addresses the core needs of the role. Minor gaps (e.g., depth in recommendation systems) prevent a perfect score.

---

### **2. Detailed Reasoning for the Score**

#### **A. Mandatory Requirements (Weight: 70%)**
| **Requirement**               | **Match (0-10)** | **Supporting Evidence**                                                                 | **Score** |
|-------------------------------|------------------|----------------------------------------------------------------------------------------|-----------|
| **Python + ML Frameworks**    | 10/10            | Lists PyTorch, TensorFlow, Keras, Scikit-learn; projects use PyTorch (AraBERT) and TensorFlow (YOLOv8). | 10        |
| **ML Model Deployment**       | 10/10            | Deployed models with **Docker + FastAPI** (50K daily requests); AWS SageMaker experience. | 10        |
| **NLP/Computer Vision**       | 10/10            | **NLP:** AraBERT fine-tuning (94% accuracy). **CV:** YOLOv8 defect detection (20% FPR reduction). | 10        |
| **Hugging Face Transformers** | 10/10            | Explicitly mentions Hugging Face; used AraBERT (a Hugging Face model) in a project.    | 10        |
| **Problem-Solving/Teamwork**  | 9/10             | Projects show **quantifiable impact** (e.g., 40% latency improvement, 15% user engagement). Teamwork implied by collaborative projects. | 9         |
| **Total (Mandatory)**         | **49/50**        | **Weighted Score: 68.6/70**                                                              |           |

#### **B. Nice-to-Have Requirements (Weight: 30%)**
| **Requirement**               | **Match (0-10)** | **Supporting Evidence**                                                                 | **Score** |
|-------------------------------|------------------|----------------------------------------------------------------------------------------|-----------|
| **Kubernetes**                | 10/10            | Lists Kubernetes under **Deployment & Cloud Computing**; likely used in production at FutureTech. | 10        |
| **Model Optimization**        | 10/10            | **Quantization** mentioned in CV (40% latency improvement).                            | 10        |
| **Recommendation Systems**    | 7/10             | Built a **Recommendation Engine** (15% user engagement boost), but lacks details on scale/complexity. | 7         |
| **Total (Nice-to-Have)**      | **27/30**        | **Weighted Score: 27/30**                                                               |           |

#### **C. Deductions (Minor Gaps)**
- **Recommendation Systems:** While Ahmed has experience, the CV doesn’t specify if the system used **collaborative filtering, deep learning (e.g., Two-Tower models), or hybrid approaches**, which are common in production.
- **AWS Depth:** AWS SageMaker is mentioned, but no details on **specific services** (e.g., Lambda, ECS, EKS) or **cost optimization**.
- **Teamwork:** The CV implies collaboration but lacks **explicit examples** of cross-functional work (e.g., with data engineers, product teams).

**Total Deductions: -8/100**

---

### **3. Key Strengths Matching the Job Requirements**
1. **End-to-End ML Expertise:**
   - **Training:** Fine-tuned AraBERT (NLP) and YOLOv8 (CV) with **state-of-the-art results**.
   - **Deployment:** Docker + FastAPI in production (50K daily requests); Kubernetes experience.
   - **Optimization:** Quantization (40% latency improvement) and model pruning (implied by "reducing model size").

2. **Specialization in NLP/CV:**
   - **NLP:** AraBERT project (dialect classification) with **94% accuracy** (12% improvement over baselines).
   - **CV:** YOLOv8 for real-time defect detection (20% FPR reduction).

3. **Hugging Face + Modern Tools:**
   - Direct experience with **Hugging Face Transformers** (AraBERT) and **PyTorch/TensorFlow**.

4. **Business Impact:**
   - **Quantifiable results** in every project/experience (e.g., 15% user engagement, 8% inventory waste reduction).

5. **Cloud + DevOps:**
   - AWS (SageMaker), Docker, Kubernetes, and CI/CD pipelines.

---

### **4. Areas Where the Candidate Falls Short**
1. **Recommendation Systems:**
   - The CV mentions a **Recommendation Engine**, but lacks details on:
     - **Approach** (collaborative filtering, content-based, hybrid, or deep learning).
     - **Scale** (e.g., number of users/items, A/B test results).
     - **Tools** (e.g., TensorFlow Recommenders, LightFM).

2. **AWS Specifics:**
   - While AWS is listed, the CV doesn’t specify **which services** were used (e.g., EKS for Kubernetes, Lambda for serverless, or S3 for storage).

3. **Team Collaboration:**
   - No explicit mention of **working with non-technical teams** (e.g., product managers, designers) or **cross-functional projects**.

4. **Model Explainability:**
   - No mention of **SHAP, LIME, or other explainability tools**, which are increasingly important in production ML.

---

### **5. Specific Examples Supporting the Evaluation**
#### **Strengths:**
- **NLP Project (AraBERT):**
  > *"Fine-tuned the AraBERT model on a dataset of over 100,000 tweets for dialect classification. Achieved 94% accuracy, outperforming baseline models by 12%."*
  - **Why it matches:** Demonstrates **Hugging Face Transformers** and **NLP expertise** with **quantifiable impact**.

- **Deployment (FutureTech):**
  > *"Deployed deep learning models to a production environment using Docker and FastAPI to serve over 50,000 requests daily."*
  - **Why it matches:** Directly addresses **Docker + FastAPI** requirement with **scalability evidence**.

- **Optimization (Quantization):**
  > *"Improved API latency by 40% by optimizing the model architecture and reducing its size (Quantization)."*
  - **Why it matches:** Shows **model optimization** (a "nice-to-have") with **measurable results**.

#### **Gaps:**
- **Recommendation Engine (FutureTech):**
  > *"Designed a Recommendation Engine that contributed to a 15% increase in user engagement during the first quarter."*
  - **Why it’s a gap:** No details on **algorithm, scale, or tools used**.

- **AWS Usage:**
  - **Missing:** Specific AWS services (e.g., "Used AWS EKS for Kubernetes orchestration" or "Deployed models on SageMaker Endpoints").

---

### **6. Final Recommendations**
- **For the Hiring Team:**
  - **Strong hire** with minimal risk. Ahmed’s **technical skills, deployment experience, and business impact** align closely with the role.
  - **Interview focus areas:**
    - Dive deeper into the **Recommendation Engine** (approach, challenges, scalability).
    - Assess **collaboration skills** (e.g., "Tell me about a time you worked with a non-technical team").
    - Explore **AWS expertise** (e.g., "Which AWS services have you used for model deployment?").

- **For the Candidate:**
  - **Improve CV by:**
    - Adding **more details** on the Recommendation Engine (e.g., "Built a hybrid recommender using LightFM and TensorFlow, serving 100K+ users").
    - Specifying **AWS services** (e.g., "Deployed models on SageMaker with Lambda for inference").
    - Including **teamwork examples** (e.g., "Collaborated with data engineers to optimize feature pipelines").

---

### **Summary**
| **Category**          | **Score** | **Notes**                                                                 |
|-----------------------|-----------|---------------------------------------------------------------------------|
| **Mandatory Skills**  | 98%       | Covers all requirements with strong evidence.                            |
| **Nice-to-Have**      | 90%       | Kubernetes and optimization are strengths; recommendation systems need depth. |
| **Business Impact**   | 100%      | Every project/experience includes quantifiable results.                  |
| **Gaps**              | 8%        | Minor: recommendation systems, AWS specifics, teamwork details.          |
| **Overall**           | **92/100**| **Top-tier candidate** with room for minor improvements in CV clarity.   |

### Extract Structured Output (Optional)

If you want to extract specific information like the score, you can parse the response.

In [13]:
# Simple extraction of score (if mentioned in response)
import re

# Try to extract score from the response
score_match = re.search(r'(\d+)\s*(?:out of 100|/100|%)', scoring_result, re.IGNORECASE)
if score_match:
    extracted_score = score_match.group(1)
    print(f"Extracted Score: {extracted_score}/100")
else:
    print("Score not found in expected format. Check the full response above.")

Extracted Score: 92/100


## Part 3: RAG-Based CV Retrieval

In this section, we'll build a RAG (Retrieval-Augmented Generation) system that:
1. Uses FAISS for fast similarity search across multiple CVs
2. Uses Mistral for intelligent re-ranking and scoring

**Hybrid Approach:** We combine the speed of vector similarity search (FAISS) with Mistral's reasoning capabilities for the best of both worlds.

### Load All CV Files

First, let's load all CV markdown files from the CVs folder.

In [ ]:
import glob
from pathlib import Path

# Find all markdown CV files
cv_base_path = Path("../datasets/CVs")
cv_files = list(cv_base_path.rglob("*.md"))

print(f"Found {len(cv_files)} CV files")

# Load all CVs
cv_data = []
for cv_file in cv_files:
    with open(cv_file, 'r', encoding='utf-8') as f:
        content = f.read()
        cv_data.append({
            'path': str(cv_file),
            'name': cv_file.stem,
            'content': content
        })

print(f"\nLoaded {len(cv_data)} CVs")
print(f"Sample CV names: {[cv['name'] for cv in cv_data[:5]]}")

Found 40 CV files

Loaded 40 CVs
Sample CV names: ['38', '18', '08', '19', '09']


### Initialize Sentence Transformer Model

**Important:** Despite the name "sentence-transformers", this library works with full documents! It processes entire CVs (even 1000+ words) and produces a single fixed-size embedding vector (e.g., 384 dimensions) that captures the semantic meaning of the whole document. This is standard practice for document similarity tasks.

In [15]:
from sentence_transformers import SentenceTransformer

# Initialize the model - works with full documents, not just sentences
# This model produces 384-dimensional embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Sentence transformer model loaded!")
print(f"Model will produce embeddings of dimension: {model.get_sentence_embedding_dimension()}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence transformer model loaded!
Model will produce embeddings of dimension: 384


### Generate Embeddings for All CVs

Each full CV (entire document) will become one embedding vector.

In [16]:
# Extract CV texts
cv_texts = [cv['content'] for cv in cv_data]

# Generate embeddings - each CV becomes one vector
# Shape will be (num_cvs, 384) - one 384-dimensional vector per CV
print("Generating embeddings for all CVs...")
cv_embeddings = model.encode(cv_texts, show_progress_bar=True)

print(f"\nEmbeddings shape: {cv_embeddings.shape}")
print(f"Number of CVs: {len(cv_data)}")
print(f"Embedding dimension: {cv_embeddings.shape[1]}")

Generating embeddings for all CVs...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]


Embeddings shape: (40, 384)
Number of CVs: 40
Embedding dimension: 384


### Generate Embedding for Job Description

Now let's create an embedding for the job description so we can find similar CVs.

In [17]:
# Generate embedding for job description
job_embedding = model.encode([job_description])

print(f"Job description embedding shape: {job_embedding.shape}")
print("Job description embedded successfully!")

Job description embedding shape: (1, 384)
Job description embedded successfully!


In [18]:
import faiss
import numpy as np

# Get the dimension of embeddings
dimension = cv_embeddings.shape[1]

# Create FAISS index using L2 (Euclidean) distance
index = faiss.IndexFlatL2(dimension)

# Convert to float32 (required by FAISS) and add embeddings
index.add(cv_embeddings.astype('float32'))

print(f"FAISS index created with {index.ntotal} CVs")
print(f"Index dimension: {dimension}")

FAISS index created with 40 CVs
Index dimension: 384


### Search for Top-K Most Relevant CVs

Now let's search for the CVs most similar to the job description using FAISS.

In [19]:
# Number of top CVs to retrieve
k = 5

# Search for top-k similar CVs
# FAISS returns distances and indices
distances, indices = index.search(job_embedding.astype('float32'), k)

print(f"Top {k} most similar CVs found:")
print(f"\nDistances (lower is more similar): {distances[0]}")
print(f"Indices: {indices[0]}")

Top 5 most similar CVs found:

Distances (lower is more similar): [0.7747874 0.8298741 0.9185155 1.0286415 1.032957 ]
Indices: [24 39 37 36 25]


### Display Retrieved CVs with Similarity Scores

Let's see which CVs were retrieved and their similarity scores.

In [20]:
# Display retrieved CVs
print("=" * 80)
for i, (idx, distance) in enumerate(zip(indices[0], distances[0]), 1):
    cv = cv_data[idx]
    print(f"\n{i}. CV: {cv['name']}")
    print(f"   Path: {cv['path']}")
    print(f"   Similarity Distance: {distance:.4f} (lower = more similar)")
    print(f"   Preview: {cv['content'][:200]}...")
    print("-" * 80)


1. CV: 01_en
   Path: ../CVs/Topic_1/01_en.md
   Similarity Distance: 0.7748 (lower = more similar)
   Preview: # Ahmed Al-Otaibi
**AI Engineer**

Riyadh, Saudi Arabia | +966500000000 | ahmed.otaibi@email.com
[linkedin.com/in/ahmed-ai] | [github.com/ahmed-code]

---

## Skills

* **Programming Languages:** Pyth...
--------------------------------------------------------------------------------

2. CV: 11
   Path: ../CVs/Topic_1/11.md
   Similarity Distance: 0.8299 (lower = more similar)
   Preview: # Nasser Al-Khaldi
**AI Engineer**

Riyadh, Saudi Arabia | +966511111111 | nasser.khaldi@email.com
[linkedin.com/in/nasser-ai] | [github.com/nasser-code]

---

## Skills

* **Programming Languages:** ...
--------------------------------------------------------------------------------

3. CV: 21
   Path: ../CVs/Topic_1/21.md
   Similarity Distance: 0.9185 (lower = more similar)
   Preview: # Yasmin Al-Jassim
**AI Engineer**

Manama, Bahrain | +97334567890 | yasmin.jassim@email.com
[linkedin.

### Use Mistral to Re-rank and Score Top Candidates

Now let's use Mistral's reasoning capabilities to intelligently re-rank and score the top candidates. This combines fast retrieval (FAISS) with intelligent evaluation (Mistral).

In [21]:
# Prepare CV summaries for Mistral
retrieved_cvs = []
for idx in indices[0]:
    cv = cv_data[idx]
    retrieved_cvs.append({
        'name': cv['name'],
        'content': cv['content'][:2000]  # Limit length for Mistral context
    })

# Create prompt for Mistral to re-rank
rerank_prompt = f"""You are an expert recruiter. I've retrieved {len(retrieved_cvs)} CVs that are similar to this job description:

Job Description:
{job_description}

Here are the retrieved CVs:

"""

for i, cv in enumerate(retrieved_cvs, 1):
    rerank_prompt += f"""
CV {i} - {cv['name']}:
{cv['content']}
---
"""

rerank_prompt += """
Please:
1. Rank these CVs from best to worst match for the job description
2. Provide a score (0-100) for each CV
3. Give brief reasoning for each ranking
4. Highlight the top 3 candidates

Format your response clearly with rankings, scores, and reasoning."""

In [22]:
# Get Mistral's re-ranking
rerank_response = client.chat.complete(
    model="mistral-large-latest",
    max_tokens=2048,
    messages=[
        {"role": "user", "content": rerank_prompt}
    ]
)

rerank_result = rerank_response.choices[0].message.content
print(rerank_result)

Here’s the ranked evaluation of the CVs based on the **AI Engineer** job description, with scores (0-100), reasoning, and highlights for the top 3 candidates:

---

### **Ranking & Scores**
| Rank | CV       | Name               | Score | Key Strengths                                                                 | Gaps                                                                 |
|------|----------|--------------------|-------|-------------------------------------------------------------------------------|----------------------------------------------------------------------|
| 1    | **01_en**| Ahmed Al-Otaibi    | **95**  | Strong Python, PyTorch/TensorFlow, NLP/CV, Hugging Face, Docker/FastAPI/AWS, model optimization, recommendation systems. | Limited Kubernetes experience (but has Docker/AWS).                 |
| 2    | **11**   | Nasser Al-Khaldi   | **88**  | TensorFlow/PyTorch, CV, Docker/Kubernetes/AWS, production deployment.         | Less NLP/Hugging Face focus; more C

### Display Final Ranked Results

Let's display Mistral's final ranking and scores:

In [23]:
# Display formatted results
display(Markdown(rerank_result))

Here’s the ranked evaluation of the CVs based on the **AI Engineer** job description, with scores (0-100), reasoning, and highlights for the top 3 candidates:

---

### **Ranking & Scores**
| Rank | CV       | Name               | Score | Key Strengths                                                                 | Gaps                                                                 |
|------|----------|--------------------|-------|-------------------------------------------------------------------------------|----------------------------------------------------------------------|
| 1    | **01_en**| Ahmed Al-Otaibi    | **95**  | Strong Python, PyTorch/TensorFlow, NLP/CV, Hugging Face, Docker/FastAPI/AWS, model optimization, recommendation systems. | Limited Kubernetes experience (but has Docker/AWS).                 |
| 2    | **11**   | Nasser Al-Khaldi   | **88**  | TensorFlow/PyTorch, CV, Docker/Kubernetes/AWS, production deployment.         | Less NLP/Hugging Face focus; more CV/predictive analytics.           |
| 3    | **21**   | Yasmin Al-Jassim   | **82**  | NLP, recommendation systems, Python, AWS/Docker.                             | Lacks Hugging Face, model optimization, and Kubernetes experience.   |
| 4    | **15**   | Sultan Al-Fahim    | **70**  | MLOps (Kubernetes, Kubeflow, CI/CD), AWS/Azure.                              | No direct ML model development (focused on infrastructure).          |
| 5    | **29**   | Faisal Al-Qahtani  | **65**  | MLOps (Kubernetes, Terraform, CI/CD), AWS.                                   | No ML model development; purely infrastructure/DevOps.               |

---

### **Detailed Reasoning**

#### **1. Ahmed Al-Otaibi (95/100) – *Top Candidate***
**Why?**
- **Perfect match** for core requirements:
  - Python + PyTorch/TensorFlow + Hugging Face (NLP project with AraBERT).
  - Production deployment (Docker, FastAPI, AWS) with **quantization** (model optimization).
  - **Recommendation systems** (nice-to-have) and **Computer Vision** (YOLOv8).
- **Strong problem-solving**: Improved API latency by 40% and reduced false positives by 20%.
- **Teamwork**: Collaborated on recommendation engines and demand forecasting.
- **Minor gap**: Kubernetes is listed but not emphasized (though Docker/AWS cover deployment).

**Standout Projects**:
- Arabic Sentiment Analysis (Hugging Face + NLP).
- Defect Detection (Computer Vision + YOLOv8).
- Recommendation Engine (15% user engagement boost).

---

#### **2. Nasser Al-Khaldi (88/100) – *Strong Contender***
**Why?**
- **Strong technical fit**:
  - PyTorch/TensorFlow, Docker/Kubernetes/AWS, and **production deployment** (edge devices with TensorFlow Lite).
  - **Computer Vision** (99.5% accuracy in defect detection) and predictive analytics.
- **Industry experience**: Worked at Saudi Aramco (ML for predictive maintenance) and SABIC (manufacturing optimization).
- **Gaps**:
  - No explicit mention of **Hugging Face** or NLP (focus is CV/predictive analytics).
  - Less emphasis on **model optimization** (quantization/pruning).

**Standout Projects**:
- Automated Quality Control (99.5% accuracy).
- Customer Churn Prediction (15% reduction in churn).

---

#### **3. Yasmin Al-Jassim (82/100) – *Good Fit for NLP/Recommendations***
**Why?**
- **NLP specialization**: Built an Arabic chatbot (Rasa) and a **recommendation system** (nice-to-have).
- **Python + AWS/Docker** for deployment.
- **Team collaboration**: Worked with marketing teams on personalized offers.
- **Gaps**:
  - No **Hugging Face** or **model optimization** (quantization/pruning).
  - No **Kubernetes** experience (only Docker/AWS).
  - Less focus on **Computer Vision**.

**Standout Projects**:
- Arabic Chatbot (50% faster response time).
- News Recommender (18% higher CTR).

---

#### **4. Sultan Al-Fahim (70/100) – *MLOps Specialist***
**Why?**
- **MLOps expertise**: Kubernetes, Kubeflow, CI/CD (Jenkins, GitLab), AWS/Azure.
- **Production experience**: Built pipelines for model monitoring and deployment.
- **Gaps**:
  - **No direct ML model development** (focused on infrastructure).
  - No mention of **Python ML frameworks** (PyTorch/TensorFlow) or **NLP/CV**.
  - **Not a fit for the core AI Engineer role** (better suited for MLOps).

**Standout Projects**:
- CI/CD Pipeline (reduced deployment time from weeks to days).
- Model Monitoring System (alerts for drift).

---

#### **5. Faisal Al-Qahtani (65/100) – *MLOps/DevOps Engineer***
**Why?**
- **Infrastructure focus**: Kubernetes, Terraform, CI/CD (GitHub Actions), AWS.
- **Production experience**: Automated model retraining pipelines (Kubeflow).
- **Gaps**:
  - **No ML model development** (purely DevOps/MLOps).
  - No **Python ML frameworks** or **NLP/CV**.
  - **Not a fit for the AI Engineer role** (better for MLOps/DevOps).

**Standout Projects**:
- Automated Model Retraining Pipeline (Kubeflow).
- Serving ML Models at Scale (Seldon Core + Kubernetes).

---

### **Top 3 Candidates**
1. **Ahmed Al-Otaibi (95/100)**
   - **Best overall fit**: Covers all core requirements + nice-to-haves (recommendation systems, model optimization).
   - **Strongest projects**: NLP (Hugging Face), CV (YOLOv8), and production deployment.

2. **Nasser Al-Khaldi (88/100)**
   - **Strong alternative**: Excellent CV/predictive analytics experience and production deployment.
   - **Industry credibility**: Worked at Saudi Aramco and SABIC.

3. **Yasmin Al-Jassim (82/100)**
   - **Best for NLP/recommendations**: Unique strength in Arabic NLP and recommendation systems.
   - **Team player**: Collaborated with marketing teams on AI solutions.

---

### **Recommendations**
- **Interview Ahmed first**: He’s the closest match and has quantifiable impact (e.g., 40% latency improvement).
- **Consider Nasser for CV-heavy roles**: If Computer Vision is a priority, he’s a strong second choice.
- **Yasmin for NLP/recommendation roles**: If the team needs Arabic NLP or recommendation systems, she’s a great fit.
- **Sultan/Faisal for MLOps roles**: If the team needs infrastructure support, these candidates are better suited for MLOps positions.

## Summary

In this notebook, we've learned:

1. **Basic Mistral AI SDK Usage**: How to initialize the client and make simple API calls
2. **CV Scoring**: How to use Mistral to evaluate a single CV against a job description
3. **RAG-Based Retrieval**: How to combine:
   - **FAISS + sentence-transformers**: Fast vector similarity search across multiple CVs
   - **Mistral API**: Intelligent re-ranking and detailed scoring of top candidates

This hybrid approach gives you the best of both worlds: speed from vector search and intelligence from Mistral's reasoning capabilities.